In [160]:
import pandas as pd
import numpy as np

####Create table with every photo

In [161]:
DIR = "/Users/damoncrockett/Desktop/cities_20k/yahoo/"

In [162]:
import glob
import os

In [163]:
for file in glob.glob(os.path.join(DIR,"*.tsv")):
    print os.path.basename(file[:-15])

bangkok
berlin
moscow
sao
tokyo


In [164]:
[os.path.basename(file[:-15])] * 5

['tokyo', 'tokyo', 'tokyo', 'tokyo', 'tokyo']

Add city names to df

In [165]:
for file in glob.glob(os.path.join(DIR,"*.tsv")):
    tmp = pd.read_csv(file)
    n = len(tmp)
    l = [os.path.basename(file[:-15])] * n
    tmp['IG_city_name'] = l
    tmp.to_csv(file,index=False)

In [166]:
df = pd.read_csv(file)

In [167]:
df.columns

Index([u'ad_id', u'Contrast', u'Symmetry', u'Top/Left Object',
       u'Middle/Left Object', u'Bottom/Left Object', u'Top/Center Object',
       u'Middle/Center Object', u'Bottom/Center Object', u'Top/Right Object', 
       ...
       u'jacket', u'purple', u'beverage', u'apparel', u'husky', u'car rally',
       u'baked', u'flowering tree', u'IG_city_name', u'local_path'],
      dtype='object', length=1069)

Add local paths to df

In [168]:
BASE = "/Users/damoncrockett/Desktop/cities_20k/"

In [169]:
for file in glob.glob(os.path.join(DIR,"*.tsv")):
    tmp = pd.read_csv(file)
    n = len(tmp.index)
    local_path = []
    for i in range(n):
        local_path.append(BASE+os.path.basename(file).split("_")[0]+"_20k/"+tmp.ad_id.loc[i]+".jpg")
    tmp['local_path'] = local_path
    tmp.to_csv(file,index=False)

Single file

In [170]:
counter = -1
for file in glob.glob(os.path.join(DIR,"*.tsv")):
    counter +=1
    if counter == 0:
        tmp = pd.read_csv(file)
    else:
        tmp = tmp.append(pd.read_csv(file))
tmp.to_csv("./all.csv",index=False)

In [171]:
df = pd.read_csv("./all.csv")

In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Columns: 1069 entries, ad_id to local_path
dtypes: float64(1066), object(3)
memory usage: 816.3+ MB


Find high-freq tags

In [173]:
l = list(df.columns)[55:1067]

In [174]:
tag = []
ct = []
for item in l:
    tag.append(item)
    # 80% because that's what Miriam used
    tmp = np.mean(d[item] > 0.8) * 100000
    ct.append(tmp)

In [175]:
tmp = pd.DataFrame(tag, columns = ['tag'])

In [176]:
tmp['ct'] = ct

In [177]:
tmp.ct = tmp.ct.apply(round)

In [178]:
tmp['per'] = tmp.ct / 5

In [179]:
np.mean(tmp.per > 200) * 1012

13.0

In [180]:
np.mean(tmp.per > 100) * 1012

22.0

In [181]:
np.mean(tmp.per > 50) * 1012

39.0

In [182]:
np.mean(tmp.per > 25) * 1012

95.0

In [183]:
slist = tmp[tmp.per >= 25]

In [184]:
len(slist)

95

In [206]:
slist.head()

,tag,ct,per
0,blackandwhite,4961,992.2
1,show,177,35.4
2,groupshot,1400,280.0
3,clothes,215,43.0
4,quilt,130,26.0


In [186]:
slist.reset_index(drop=True,inplace=True)

Subset 'df' to these columns, plus the style columns

In [187]:
tags = list(slist.tag)

In [188]:
style = list(df.columns[:55])

In [189]:
last = ['IG_city_name','local_path']

In [190]:
cols = last+style+tags

In [191]:
sub = df[cols]

In [192]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Columns: 152 entries, IG_city_name to beverage
dtypes: float64(149), object(3)
memory usage: 116.7+ MB


Because R does not play well with whitespace, we will get rid of it

In [194]:
tmp = list(slist.tag)
tmp = [item.translate(None," ") for item in tmp]
slist.tag = tmp

In [195]:
tmp = list(sub.columns)
tmp = [item.translate(None," /)(") for item in tmp]
sub.columns = tmp

In [196]:
slist.to_csv("./tags_by_features.csv",index=False)

In [197]:
sub.to_csv("./tags_by_features_data.csv",index=False)

####Sending to R for variance and ANOVA...

####...and we're back

In [212]:
var_table = pd.read_csv("./var_table.csv")
del var_table['Unnamed: 0']

In [213]:
rsq_table = pd.read_csv("./var_table_rsq.csv")
del rsq_table['Unnamed: 0']

In [225]:
features = list(rsq_table.columns[:54])

####Now we can look for high city-variance features by object tag

In [302]:
cutoff = 0.1
top = 0.4

In [303]:
df = pd.DataFrame()
n = len(rsq_table)
all_feats = []
for i in range(n):
    feats = []
    for feature in features:
        tmp = rsq_table[feature].loc[i]
        if cutoff <= tmp < top:
            feats.append(feature)
    all_feats.append(feats)
df['tag'] = rsq_table.tag
df['feats'] = all_feats

In [263]:
# Our general approach is this:
# We want to make, for each tag/feature
# combo in our set, a visualization 
# with 5 clusters, one for each city.
# They can be combined later into 
# larger plots, using GIMP.

In [304]:
TARGET = "/Users/damoncrockett/Desktop/cities_20k/viz/city-tag/"

In [305]:
cities = ['bangkok', 'berlin', 'moscow', 'sao', 'tokyo']

In [306]:
from shapely.geometry import Point
from PIL import Image

In [307]:
thumb_side = 64

In [308]:
n = len(df)

In [309]:
for i in range(n):
    tag = df.tag.loc[i]
    tag_subset = sub[sub[tag] >= 0.8]
    for feat in df.feats.loc[i]:
        feat_subset = tag_subset[[feat,'IG_city_name','local_path']]
        max_count = 0
        for city in cities:
            tmp = feat_subset[feat_subset.IG_city_name==city]
            if len(tmp) > max_count:
                max_count = len(tmp)
        entourage_side = int(round(np.sqrt(max_count)))
        h = entourage_side + 4
        w = entourage_side * 5 + 4
        
        x,y = range(w) * h, np.repeat(range(h),w)
        grid_list = pd.DataFrame(x,columns=['x'])
        grid_list['y'] = y
        
        point = []
        m = len(grid_list.index)
        for i in range(m):
            point.append(Point(grid_list.x.loc[i],grid_list.y.loc[i]))
        grid_list['point'] = point
        open_grid = list(grid_list.point)
        
        exemplar_h = int(round(h/2))
        moscow = int(round(w/2))
        berlin = moscow - entourage_side
        bangkok = berlin - entourage_side
        sao = moscow + entourage_side
        tokyo = sao + entourage_side
        
        d = {
        'moscow':int(round(w/2)),
        'berlin':moscow - entourage_side,
        'bangkok':berlin - entourage_side,
        'sao':moscow + entourage_side,
        'tokyo':sao + entourage_side
        }
        
        canvas = Image.new('RGB',(w * thumb_side, h * thumb_side),(50,50,50))

        for city in cities:
            exemplar = Point(d[city],exemplar_h)
            city_df = feat_subset[feat_subset.IG_city_name==city]
            city_df.sort(feat,inplace=True,ascending=False)
            city_df.reset_index(drop=True,inplace=True)
            im = Image.open(city_df.local_path.loc[0])
            im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
            x = int(exemplar.x) * thumb_side
            y = int(exemplar.y) * thumb_side
            canvas.paste(im,(x,y))
            open_grid.remove(exemplar)
            
            for i in range(1,len(city_df)):
                im = Image.open(city_df.local_path.loc[i])
                im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
                closest_open = min(open_grid,key=lambda x: exemplar.distance(x))
                x = int(closest_open.x) * thumb_side
                y = int(closest_open.y) * thumb_side
                canvas.paste(im,(x,y))
                open_grid.remove(closest_open)
        canvas.save(TARGET+tag+"_"+feat+".png")